# `keras` code for last-man-standing hackathon by AnalyticsVidhya
### Code created on : 31st Jan 2015 | Author: Bargava

Link to competition
http://datahack.analyticsvidhya.com/contest/last-man-standing

#### Solution Approach
In this, we will try out the deep learning library `keras` 

In [1]:
#Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import StratifiedShuffleSplit
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD

Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.


/Users/barsubra/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
#read train, test and samplesub datasets
train = pd.read_csv("../data/trainUpdated.csv")
test = pd.read_csv("../data/testUpdated.csv")
samplesub = pd.read_csv("../data/samplesubUpdated.csv")
label = pd.read_csv("../data/labelsUpdated.csv")

In [3]:
#Convert the inputs to numpy array
train = np.array(train)
test = np.array(test)
label = np.array(label)

In [8]:
label[label==2] = 0

In [9]:
#Find the list of indices for first layer and second layer stacking
validation_indices = StratifiedShuffleSplit(label, test_size=0.85, n_iter=1)

In [10]:
for train_index, validation_index in validation_indices:
    train_data, validation_data = train[train_index], train[validation_index] 
    train_label, validation_label = label[train_index], label[validation_index]

In [11]:
#Let's check and find out proportions of the classes in train, train_data and validation_data
#This is to see if stratified sampling has happened properly
print "train:", np.unique(label, return_counts=True)[1].astype(float)/label.shape[0]
print "train data:",  np.unique(train_label, return_counts=True)[1].astype(float)/train_label.shape[0]
print "validation train:",  np.unique(validation_label, return_counts=True)[1].astype(float)/validation_label.shape[0]

train: [ 0.86149812  0.13850188]
train data: [ 0.8614946  0.1385054]
validation train: [ 0.86149874  0.13850126]


In [12]:
print train.shape, train_data.shape, validation_data.shape

(88858, 12) (13328, 12) (75530, 12)


# `keras`

In [13]:
np.unique(validation_label, return_counts=True)

(array([0, 1]), array([65069, 10461]))

In [32]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(576, input_dim=12, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(288, init='uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(144))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(1,init='uniform'))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

In [33]:
model.compile(loss='binary_crossentropy', optimizer=sgd)

In [34]:
model.fit(validation_data, validation_label, batch_size=24, nb_epoch=12)

Epoch 1/12
75530/75530 [==============================] - 13s - loss: 13.7343    
Epoch 2/12
42360/75530 [===============>..............] - ETA: 6s - loss: 13.7486

KeyboardInterrupt: 

In [ ]:
train_data.shape

In [ ]:
score = model.evaluate(train_data, train_label, batch_size=12)
score

In [23]:
preds = model.predict_classes(test)

59310/59310 [==============================] - 1s     


In [24]:
np.unique(preds, return_counts=True)

(array([0]), array([59310]))

In [25]:
?np.expm1